In [1]:
# Configurations
import torch
from datetime import datetime
import logging
from pathlib import Path
import os


In [2]:
# connect to google colab
from google.colab import drive
drive.mount("/content/gdrive")

ModuleNotFoundError: No module named 'google'

In [4]:
# MPS doesn't work for this project...

# Configure device: CUDA, MPS, CPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA as device")
else:
    # Check that MPS is available
    if not torch.backends.mps.is_available():
        if not torch.backends.mps.is_built():
            print("MPS not available because the current PyTorch install was not "
                  "built with MPS enabled.")
        else:
            print("MPS not available because the current MacOS version is not 12.3+ "
                  "and/or you do not have an MPS-enabled device on this machine.")
        device = torch.device("cpu")
        print("Using CPU as device")
    else:
        device = torch.device("mps")
        print("Using MPS as device")


torch.set_default_device(device)

MPS not available because the current PyTorch install was not built with MPS enabled.
Using CPU as device


In [5]:
# Configure Directory
project_dir = Path("/content/gdrive/MyDrive/Colab Notebooks/Food-Classifier")
rawdata_dir = project_dir / "rawdata"
data_dir = project_dir / "data"
model_dir = project_dir / "models"
log_dir = project_dir / "logs"

rawdata_dir.mkdir(parents=True, exist_ok=True)
data_dir.mkdir(parents=True, exist_ok=True)
model_dir.mkdir(parents=True, exist_ok=True)
log_dir.mkdir(parents=True, exist_ok=True)

print(f'project_dir: {project_dir}')
print(f'rawdata_dir: {rawdata_dir}')
print(f'data_dir: {data_dir}')
print(f'model_dir: {model_dir}')
print(f'log_dir: {log_dir}')

project_dir: /content/gdrive/MyDrive/Colab Notebooks/Food-Classifier
rawdata_dir: /content/gdrive/MyDrive/Colab Notebooks/Food-Classifier/rawdata
data_dir: /content/gdrive/MyDrive/Colab Notebooks/Food-Classifier/data
model_dir: /content/gdrive/MyDrive/Colab Notebooks/Food-Classifier/models
log_dir: /content/gdrive/MyDrive/Colab Notebooks/Food-Classifier/logs


In [6]:
# Configure Logger
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
log_file = log_dir / f'log_{timestamp}.log'

logger = logging.getLogger('transformer_log')
logger.setLevel(logging.INFO)

file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)

In [7]:
# Define input image size 
# We are going to resize the original image
EPOCHS = 100 
BATCH_SIZE = 1
IMG_SIZE = 256
PATCH_SIZE = 4 
IN_CHANNELS = 3 
N_HEAD = 8
D_MODEL = 400 
FFN_HIDDEN = 512 
MLP_HIDDEN = 1024 
N_LAYERS = 6 
CLASS_NUM = 101
DROP_PROB = 0.1 
INIT_LR = 0.01
NUM_WORKERS=2

torch.multiprocessing.set_start_method('spawn')

In [8]:
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torch import Generator

In [9]:
# Define Image transform function
img_transform = transforms.Compose([
    transforms.RandomResizedCrop((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

In [10]:
# Prepare Food101 dataset
train_data = datasets.Food101(root=rawdata_dir, split="train", download=True, transform=img_transform)
test_data = datasets.Food101(root=rawdata_dir, split="test", download=True, transform=img_transform)

100%|██████████| 5.00G/5.00G [04:32<00:00, 18.4MB/s]


Extracting /content/gdrive/MyDrive/Colab Notebooks/Food-Classifier/rawdata/food-101.tar.gz to /content/gdrive/MyDrive/Colab Notebooks/Food-Classifier/rawdata


In [11]:
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, pin_memory=True, generator=torch.Generator(device=device), num_workers=NUM_WORKERS)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, drop_last=True, pin_memory=True, generator=torch.Generator(device=device), num_workers=NUM_WORKERS)

In [ ]:
# Visual Transformer Implementation
from torch import nn
import torch
import math

In [ ]:
## Patch Embeddings
class PatchEmbedding(nn.Module):
    """
    split image into grids, and convert to a latent space vector
    """

    def __init__(self, image_size, patch_size, in_channels, d_model, device):
        super().__init__()
        self.image_size = image_size
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.d_model = d_model
        # image_size should be dividable by patch_size
        # default: image_size=400, patch_size=40
        self.num_patches = (self.image_size // self.patch_size) ** 2

        # using convolution to create non-overlapping patches
        self.emb = nn.Conv2d(self.in_channels, self.d_model, kernel_size=self.patch_size, stride=self.patch_size, device=device)

    def forward(self, x):
        # input shape: (batch_size, in_channels, image_size, image_size)
        # output shape: (batch_size, num_patches, out_channels)

        x = self.emb(x)
        # intermediate x's shape: (batch_size, seq_len, sqrt(num_patches), sqrt(num_patches))

        x = x.flatten(2).transpose(1, 2)

        return x

## Positional Embeddings
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, image_size, patch_size, device):
        super().__init__()
        num_patches = (image_size // patch_size) ** 2
        self.max_len = num_patches + 1
        self.d_model = d_model

        self.encoding = torch.zeros(self.max_len, d_model, device=device)

        # sinusoid positional embedding is static, which doesn't require training
        # we turn off gradient engine to reduce memory in GPU
        self.encoding.requires_grad = False

        pos = torch.arange(0, self.max_len, device=device)
        pos = pos.float().unsqueeze(dim=1)
        # shape of pos: (max_len, 1)

        _2i = torch.arange(0, self.d_model, step=2, device=device).float()
        # shape of _2i: (num_patches // 2, )

        self.encoding[:, 0::2] = torch.sin(pos / (10000 ** (_2i / self.d_model)))
        self.encoding[:, 1::2] = torch.cos(pos / (10000 ** (_2i / self.d_model)))

    def forward(self, x):
        _, seq_len, _ = x.shape
        return self.encoding[:seq_len, :]


class TransformerEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, d_model, drop_prob, device):
        super().__init__()
        self.patch_emb = PatchEmbedding(image_size, patch_size, in_channels, d_model, device)
        self.pos_emb = PositionalEmbedding(d_model, image_size, patch_size, device)
        self.dropout = nn.Dropout(p=drop_prob)

        # Similar to BERT model, we should add CLS token to the starting of the sequence
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))

    def forward(self, x):
        batch_size, _, _, _ = x.shape

        x = self.patch_emb(x)


        cls_token = self.cls_token.expand(batch_size, -1, -1)
        # cls_token shape: (batch_size, 1, d_model)

        x = torch.cat((cls_token, x), dim=1)
        pos_emb = self.pos_emb(x)

        return self.dropout(x + pos_emb)

In [ ]:
## Attention Block
class SelfAttentionBlock(nn.Module):
    """
    A attention block with scale dot product attention for
    Query, Key, Value
    """
    def __init__(self):
        super().__init__()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v, eps=1e-12):
        # Since this is not a NLP, we don't need any padding-mask or look-ahead mask
        # also, q, k, v have the same shape
        batch_size, n_head, seq_len, d_tensor = k.shape

        k_T = k.transpose(2, 3)
        att_weight = (q @ k_T) / math.sqrt(d_tensor)

        # att_weight shape: batch_size, n_head, seq_len, d_tensor
        att_score = self.softmax(att_weight)

        return att_score @ v, att_score

## Multihead Attention Block
class MultiheadAttentionBlock(nn.Module):
    def __init__(self, n_head, d_model):
        super().__init__()
        self.n_head = n_head
        self.d_model = d_model

        self.Wq = nn.Linear(d_model, d_model)
        self.Wk = nn.Linear(d_model, d_model)
        self.Wv = nn.Linear(d_model, d_model)

        self.attention = SelfAttentionBlock()

        self.Wconcat = nn.Linear(d_model, d_model)

    def split(self, tensor):
        """
        split tensor into n_heads
        (batch_size, seq_len, d_model) -> (batch_size, n_head, seq_len, d_tensor)
        """
        batch_size, seq_len, d_model = tensor.shape

        d_tensor = d_model // self.n_head

        tensor = tensor.reshape(batch_size, seq_len, self.n_head, d_tensor).transpose(1, 2)

        return tensor

    def concat(self, tensor):
        """
        reverse of split
        (batch_size, n_head, seq_len, d_tensor) -> (batch_size, seq_len, d_model)
        """
        batch_size, n_head, seq_len, d_tensor = tensor.shape

        tensor = tensor.transpose(1, 2).reshape(batch_size, seq_len, n_head * d_tensor)

        return tensor

    def forward(self, x):

        # apply Wq, Wk, Wv to get q, k, v
        query = self.split(self.Wq(x))
        key = self.split(self.Wk(x))
        value = self.split(self.Wv(x))

        # apply attention
        out, attn_weight = self.attention(query, key, value)

        out = self.concat(out)

        out = self.Wconcat(out)

        return out, attn_weight


In [ ]:
## Define FeedForward Network
class FeedForwardBlock(nn.Module):
    def __init__(self, d_model, ffn_hidden, drop_prob=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, ffn_hidden)
        self.linear2 = nn.Linear(ffn_hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [ ]:
# Define Transformer EncoderBlock and Encoder
class EncoderBlock(nn.Module):
    def __init__(self, n_head, d_model, ffn_hidden, drop_prob=0.1):
        super().__init__()
        self.norm = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(p=drop_prob)

        self.multihead_attn = MultiheadAttentionBlock(n_head, d_model)
        self.ffn = FeedForwardBlock(d_model, ffn_hidden, drop_prob)
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x):
        residual = x

        x = self.norm(x)
        x, attn_weight = self.multihead_attn(x)

        x = self.dropout1(x + residual)

        residual = x

        x = self.norm(x)
        x = self.ffn(x)

        x = self.dropout2(x + residual)

        return x, attn_weight

class Encoder(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, n_head, d_model, ffn_hidden, n_layers, device,  drop_prob=0.1):
        super().__init__()
        self.emb = TransformerEmbedding(image_size, patch_size, in_channels, d_model, drop_prob, device)
        self.layers = nn.ModuleList([EncoderBlock(n_head, d_model, ffn_hidden, drop_prob)])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        x = self.emb(x)

        attn_weights = []

        for layer in self.layers:
            x, attn_weight = layer(x)
            attn_weights.append(attn_weight)

        x = self.norm(x)
        return x, attn_weights

In [ ]:
## Define MLP head for final classification
class MLPHead(nn.Module):
    def __init__(self, d_model, class_num, mlp_hidden, drop_prob):
        super().__init__()
        self.linear1 = nn.Linear(d_model, mlp_hidden)
        self.gelu = nn.GELU()
        self.linear2 = nn.Linear(mlp_hidden, class_num)
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.gelu(x)

        x = self.linear2(x)
        x = self.dropout(x)
        return x

In [ ]:
class ViT(nn.Module):
    """
    ViT Model
    """
    def __init__(self, image_size, patch_size, in_channels, n_head, d_model, ffn_hidden, mlp_hidden, n_layers, class_num, device, drop_prob=0.1):
        super().__init__()
        seq_len = (image_size // patch_size) ** 2 + 1
        self.encoder = Encoder(image_size, patch_size, in_channels, n_head, d_model, ffn_hidden, n_layers, device, drop_prob)
        self.mlp_head = MLPHead(d_model, class_num, mlp_hidden, drop_prob)

    def forward(self, img):
        x, attn_weights = self.encoder(img)

        logits = self.mlp_head(x[:, 0, :])

        return (logits, attn_weights)


In [ ]:
# Train the Vit using Food101 dataset

In [ ]:
from torch.optim import Adam
from datetime import datetime
import torch
from tqdm import tqdm

In [ ]:
model = ViT(
    image_size=IMG_SIZE,
    patch_size=PATCH_SIZE,
    in_channels=IN_CHANNELS,
    n_head=N_HEAD,
    d_model=D_MODEL,
    ffn_hidden=FFN_HIDDEN,
    mlp_hidden=MLP_HIDDEN,
    n_layers=N_LAYERS,
    class_num=CLASS_NUM,
    device=device,
    drop_prob=DROP_PROB,
)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

logger.info(f'model parameter #: {count_parameters(model)}')

In [ ]:
# Setup optimizer
optimizer = Adam(params = model.parameters(), lr=INIT_LR)

# Setup loss function for training
loss_func = nn.CrossEntropyLoss()

In [ ]:
train_loss_per_epoch = []
test_loss_per_epoch = []

In [ ]:
def train_epoch(epoch_num): 
    # Prepare recording CUDA memory snapshot
    torch.cuda.memory._record_memory_history(
        max_entries=MAX_NUM_OF_MEM_EVENTS_PER_SNAPSHOT
    )
    model.train()
    train_epoch_loss = 0 

    for step, (img, food) in tqdm(enumerate(train_dataloader)):
        img = img.to(device)
        food = food.to(device)
        out, _ = model(img)

        loss = loss_func(out, food)

        
        loss.backward()

        optimizer.step()
        train_epoch_loss += loss.item()

        if step % 100 == 0: 
            logger.info(f'EPOCH #{epoch_num} STEP #{step} | loss: {loss.item()}, avg_loss: {train_epoch_loss / (step + 1)}')

        if step == 500:
            try:
                memory_dump_path = log_dir / f'memory_dump_{timestamp}.log'
                torch.cuda.memory._dump_snapshot()
                # stop recording memory history
                torch.cuda.memory._record_memory_history(enabled=None)
            except Exception as e:
                logger.error(f'Failed to capture memory snapshot {e}')
            
        optimizer.zero_grad()

    train_step_loss = train_epoch_loss / (step + 1) 
    return train_epoch_loss 

In [ ]:
def evaluate():
    model.eval()
    test_epoch_loss = 0

    with torch.no_grad():
        for step, (img, food) in tqdm(enumerate(test_dataloader)):
            img = img.to(device)
            food = food.to(device)
            out, _ = model(img)

            loss = loss_func(out, food)

            test_epoch_loss += loss.item()

    test_step_loss = test_epoch_loss / (step + 1)

    return test_epoch_loss

In [ ]:
# Actual training is done here

for epoch in range(EPOCHS):
    train_epoch_loss, train_step_loss = train_epoch(epoch)
    test_epoch_loss, test_step_loss = evaluate()

    train_loss_per_epoch.append(train_epoch_loss)
    test_loss_per_epoch.append(test_epoch_loss)

    logger.info(f'Epoch #{epoch} End | Train Loss: {train_epoch_loss} | Test Loss: {test_epoch_loss}')

    model_path = model_dir / f'model_{timestamp}_{epoch}'
    torch.save(model.state_dict(), model_path)

logger.info(f'Training Completely Ended!!')
